In [1]:
import cv2
import os

from transformers import AutoTokenizer
from vllm import LLM, SamplingParams
from vllm.assets.image import ImageAsset 
from vllm.assets.video import VideoAsset
from vllm.utils import FlexibleArgumentParser
# Specify the maximum number of frames per video to be 4. This can be changed.
# llm = LLM("Qwen/Qwen2.5-VL-3B-Instruct", limit_mm_per_prompt={"image": 4}, max_model_len=10000)
llm = LLM(
        model="Qwen/Qwen2.5-VL-7B-Instruct",
        max_model_len=8000,
        max_num_seqs=10)


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-02-21 06:41:48.568817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740120108.577052  272289 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740120108.580770  272289 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 02-21 06:41:50 __init__.py:207] Automatically detected platform cuda.
INFO 02-21 06:41:57 config.py:549] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 02-21 06:41:57 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.4.dev3+g6a417b86.d20250220) with config: model='Qwen/Qwen2.5-VL-7B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-VL-7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, co

Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


INFO 02-21 06:42:53 model_runner.py:1115] Loading model weights took 15.6270 GB


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


WARNING 02-21 06:42:54 model_runner.py:1288] Computed max_num_seqs (min(10, 8000 // 32768)) to be less than 1. Setting it to the minimum value of 1.


It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.


WARNING 02-21 06:42:57 profiling.py:192] The context length (8000) of the model is too short to hold the multi-modal embeddings in the worst case (32768 tokens in total, out of which {'image': 16384, 'video': 16384} are reserved for multi-modal embeddings). This may cause certain multi-modal inputs to fail during inference, even when the input text is short. To avoid this, you should increase `max_model_len`, reduce `max_num_seqs`, and/or reduce `mm_counts`.
INFO 02-21 06:42:58 worker.py:267] Memory profiling takes 4.20 seconds
INFO 02-21 06:42:58 worker.py:267] the current vLLM instance can use total_gpu_memory (94.50GiB) x gpu_memory_utilization (0.90) = 85.05GiB
INFO 02-21 06:42:58 worker.py:267] model weights take 15.63GiB; non_torch_memory takes 0.15GiB; PyTorch activation peak memory takes 1.09GiB; the rest of the memory reserved for KV Cache is 68.18GiB.
INFO 02-21 06:42:58 executor_base.py:111] # cuda blocks: 79788, # CPU blocks: 4681
INFO 02-21 06:42:58 executor_base.py:116] M

Capturing CUDA graph shapes: 100%|██████████| 5/5 [00:02<00:00,  2.01it/s]

INFO 02-21 06:43:06 model_runner.py:1562] Graph capturing finished in 3 secs, took 0.17 GiB
INFO 02-21 06:43:06 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 12.85 seconds


In [2]:
sampling_params = SamplingParams(temperature=0.7,
                                     max_tokens=256)

In [3]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("HuggingFaceFV/finevideo", split="train")

# Access the first video
video = ds[0]["mp4"]

#full dataset (600GB of data)
#dataset = load_dataset("HuggingFaceFV/finevideo/", split="train", num_proc=24)


Resolving data files:   0%|          | 0/1357 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1357 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/907 [00:00<?, ?it/s]

In [4]:
# 1. Specify a dataset of video (100 videos)
# 2. Specify a prompt that asks the model to generate a description of the video * 100
# 3. You should be able to feed all of it together at once to the model to generate (DO NOT FEED ONE BY ONE)

In [5]:
from decord import VideoReader, cpu
from PIL import Image
import matplotlib.pyplot as plt 
from tqdm import tqdm
import numpy as np 

In [6]:

# Load the videos 
dataset_path = "/home/ubuntu/temp/10k_vid_dataset/mp4s"
os.makedirs(dataset_path, exist_ok=True)

num_videos = 1500
videos = []
num_chunks = 3
time_per_chunk = 10
for i in tqdm(range(num_videos), total=num_videos):
    #save video 
    video_path = os.path.join(dataset_path, f"video_{i}.mp4")
    with open(video_path, "wb") as f:
        f.write(ds[i]["mp4"])
    # read video    
    vr = VideoReader(video_path, ctx=cpu(0))
    vid_dict = {"vid_path": video_path}
    success = True
    for chunk_idx in range(num_chunks):
        try:
            frame_idxs = np.arange(chunk_idx * time_per_chunk * 30, (chunk_idx + 1) * time_per_chunk * 30, 30)
            nd_frames = vr.get_batch(frame_idxs).asnumpy()
            vid_dict[f"vT{chunk_idx}"] = nd_frames
        except:
            print(f"Error processing")
            success = False
    if(success):
        videos.append(vid_dict)

  2%|▏         | 33/1500 [00:03<02:23, 10.22it/s]

Error processing
Error processing


  5%|▍         | 69/1500 [00:08<02:59,  7.99it/s][h264 @ 0xbc1179457f00] mmco: unref short failure
[h264 @ 0xbc1179457f00] mmco: unref short failure
  7%|▋         | 107/1500 [00:13<02:52,  8.06it/s]

Error processing


  9%|▉         | 136/1500 [00:16<02:36,  8.69it/s]

...

 99%|█████████▊| 1480/1500 [03:12<00:02,  6.67it/s]

Error processing


 99%|█████████▉| 1492/1500 [03:14<00:00,  8.83it/s]

Error processing


100%|██████████| 1500/1500 [03:15<00:00,  7.67it/s]


In [7]:
# DATA STRUCTURE
# videos = [{"vid_path": "path...",
#            "vid": [frame1, frame2, frame3, ...],
#            "T1": {"description": {"Q": "text...", "A": "text..."},
#                   "temporal": {"time": int, "content": "text..."}},
#                   "question": {"Q": "text...", "A": "text..."},
#           }]

In [8]:
from vllm.assets.video import VideoAsset

def generate_video_description(video_frame_list, prompt, num_chunks):
    messages = []
    for idx, dict_obj in enumerate(video_frame_list):
        for chunk_idx in range(num_chunks):
            if(type(prompt) == list):
                prompt_curr = prompt[idx*num_chunks + chunk_idx]
            else:
                prompt_curr = prompt
            message = {"prompt": prompt_curr, "multi_modal_data": {"video": dict_obj[f"vT{chunk_idx}"]}}
            messages.append(message)
    # Perform inference and log output.
    print("Starting inference on ", len(messages), " messages")
    outputs = llm.generate(messages, sampling_params=sampling_params)
    generated_texts = []
    for idx in range(len(outputs)):
        generated_text = outputs[idx].outputs[0].text
        generated_texts.append(generated_text)
    print("Generated", len(generated_texts), "texts")
    return generated_texts
    

# Description Generation

In [9]:
prompt_describe = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              f"Describe what is shown in the video. <|im_end|>\n"
              "<|im_start|>assistant\n")

generated_texts = generate_video_description(videos, prompt_describe, num_chunks)

for idx, text in enumerate(generated_texts):
    chunk_idx = idx%num_chunks
    list_idx = idx//num_chunks
    start_time = chunk_idx * time_per_chunk
    end_time = (chunk_idx + 1) * time_per_chunk
    print(f"Video {list_idx} chunk {chunk_idx} time {start_time} to {end_time} seconds")
    videos[list_idx][f"T{chunk_idx}_description"] = {"Q": f"What occurs between {start_time} and {end_time} seconds?", "A": text}

Starting inference on  4356  messages


Processed prompts: 100%|██████████| 4356/4356 [16:25<00:00,  4.42it/s, est. speed input: 6452.11 toks/s, output: 683.52 toks/s]

Generated 4356 texts
Video 0 chunk 0 time 0 to 10 seconds
Video 0 chunk 1 time 10 to 20 seconds
Video 0 chunk 2 time 20 to 30 seconds
Video 1 chunk 0 time 0 to 10 seconds
Video 1 chunk 1 time 10 to 20 seconds
Video 1 chunk 2 time 20 to 30 seconds
Video 2 chunk 0 time 0 to 10 seconds
Video 2 chunk 1 time 10 to 20 seconds
Video 2 chunk 2 time 20 to 30 seconds
Video 3 chunk 0 time 0 to 10 seconds
Video 3 chunk 1 time 10 to 20 seconds
Video 3 chunk 2 time 20 to 30 seconds
Video 4 chunk 0 time 0 to 10 seconds
Video 4 chunk 1 time 10 to 20 seconds
Video 4 chunk 2 time 20 to 30 seconds
Video 5 chunk 0 time 0 to 10 seconds
Video 5 chunk 1 time 10 to 20 seconds
Video 5 chunk 2 time 20 to 30 seconds
Video 6 chunk 0 time 0 to 10 seconds
Video 6 chunk 1 time 10 to 20 seconds
Video 6 chunk 2 time 20 to 30 seconds
Video 7 chunk 0 time 0 to 10 seconds
Video 7 chunk 1 time 10 to 20 seconds
Video 7 chunk 2 time 20 to 30 seconds
Video 8 chunk 0 time 0 to 10 seconds
Video 8 chunk 1 time 10 to 20 seconds


In [10]:
print(videos[-1]["vid_path"])
for key in videos[-1].keys():
    if("description" in key):
        print(f"{key}: {videos[-1][key]}")

/home/ubuntu/temp/10k_vid_dataset/mp4s/video_1499.mp4
T0_description: {'Q': 'What occurs between 0 and 10 seconds?', 'A': 'The image is a static frame with text displayed on a blue background. The text appears to be an announcement or notice, which includes the following information:\n\n- "Overpasses for Obama\'s Impeachment"\n- "Sat. Aug 17, 2013"\n- "St. Charles, Missouri"\n- "Two Patriots Arrested by MO Highway Patrolman Jenkins for refusing to leave when \'requested\' to leave"\n\nIt seems to be a report about an event that occurred on a specific date, involving two individuals referred to as "Patriots" who were arrested by a Missouri Highway Patrolman named Jenkins, allegedly for refusing to leave a location after being asked to do so. The context suggests a political demonstration or rally is being discussed.'}
T1_description: {'Q': 'What occurs between 10 and 20 seconds?', 'A': 'The video begins with a blue screen containing white text that reads, "Jenkins \'requested\' to leave

# Time Step Generation

In [11]:
prompt_timestep = ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              "What occurs at time {time} seconds?<|im_end|>\n"
              "<|im_start|>assistant\n")

sample_time_steps = np.random.randint(0, time_per_chunk, num_videos*num_chunks)

prompt_list_timesteps = []
for time_step in sample_time_steps:
    prompt = prompt_timestep.replace("{time}", str(time_step))
    prompt_list_timesteps.append(prompt)

generated_texts = generate_video_description(videos, prompt_list_timesteps, num_chunks)

for idx, text in enumerate(generated_texts):
    chunk_idx = idx%num_chunks
    list_idx = idx//num_chunks
    vid_time = sample_time_steps[idx]
    calc_time = chunk_idx*time_per_chunk+sample_time_steps[idx] # have to recalculate due to chunking
    videos[list_idx][f"T{chunk_idx}_timestep"] = {"Q:": f"What occurs at time {calc_time} seconds?", "A": text.replace(str(vid_time), str(calc_time))}

Starting inference on  4356  messages


Processed prompts: 100%|██████████| 4356/4356 [11:04<00:00,  6.55it/s, est. speed input: 9557.34 toks/s, output: 468.37 toks/s]

Generated 4356 texts


In [12]:
print(videos[0].keys())
for key in videos[0].keys():
    if("timestep" in key):
        print(f"{key}: {videos[0][key]}")

dict_keys(['vid_path', 'vT0', 'vT1', 'vT2', 'T0_description', 'T1_description', 'T2_description', 'T0_timestep', 'T1_timestep', 'T2_timestep'])
T0_timestep: {'Q:': 'What occurs at time 5 seconds?', 'A': 'At time 5 seconds, the video transitions from a sign that reads "Heavy Equipment Operator Training Centre" to a view of a construction site.'}
T1_timestep: {'Q:': 'What occurs at time 10 seconds?', 'A': "At time 10 seconds, the video starts with a scene showing a woman named Sara Paynton, who is identified as a SICA/Velocity client. The background suggests that she is at a construction site, as there are construction vehicles and equipment visible. This indicates that the video may be related to construction, possibly showcasing a project or a company's work in the construction industry."}
T2_timestep: {'Q:': 'What occurs at time 23 seconds?', 'A': 'At time 23 seconds in the video, the large yellow dump truck is shown driving on a dirt road, which is part of the construction site. The 

# QA

In [13]:
prompt_gen_question =  ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              f"What is a vision understanding question you can ask about the visual content of the video. Only output the question<|im_end|>\n"
              "<|im_start|>assistant\n")

generated_texts = generate_video_description(videos, prompt_gen_question, num_chunks)

questions = generated_texts

Starting inference on  4356  messages


Processed prompts: 100%|██████████| 4356/4356 [07:33<00:00,  9.61it/s, est. speed input: 14139.70 toks/s, output: 165.28 toks/s]

Generated 4356 texts


In [14]:
prompt_answer_question =  ("<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n"
              f"<|im_start|>user\n<|vision_start|><|video_pad|><|vision_end|>"
              "{question}<|im_end|>\n"
              "<|im_start|>assistant\n")


prompt_qas = []
for q in questions:
    qa = prompt_answer_question.replace("{question}", q)
    prompt_qas.append(qa)


generated_texts = generate_video_description(videos, prompt_qas, num_chunks)
answers = generated_texts

for idx in range(len(answers)):
    chunk_idx = idx%num_chunks
    list_idx = idx//num_chunks
    q = questions[idx]
    a = answers[idx]
    videos[list_idx][f"T{chunk_idx}_qa"] = {"Q": q, "A": a}

Starting inference on  4356  messages


Processed prompts: 100%|██████████| 4356/4356 [12:35<00:00,  5.76it/s, est. speed input: 8451.85 toks/s, output: 544.34 toks/s]

Generated 4356 texts


In [15]:
vid_set = videos[-3]
print(vid_set.keys())
for key in vid_set.keys():
    if("qa" in key):
        print("Q", vid_set[key]["Q"])
        print("A", vid_set[key]["A"])
        print("______________")

dict_keys(['vid_path', 'vT0', 'vT1', 'vT2', 'T0_description', 'T1_description', 'T2_description', 'T0_timestep', 'T1_timestep', 'T2_timestep', 'T0_qa', 'T1_qa', 'T2_qa'])
Q What is the woman doing at the subway station in the video?
A In the video, the woman is seen navigating the subway station. She checks her phone while walking through the station, and then proceeds to a turnstile to scan her transit card. This indicates that she is likely preparing to enter the subway system.
______________
Q What is the station name displayed on the train's sign?
A The station name displayed on the train's sign is "15th Street."
______________
Q What is the purpose of the event taking place in the parking lot?
A The event in the parking lot appears to be a tailgate party before a sports game. Tailgate parties are common before sporting events, especially football games, where people gather to socialize, enjoy food, and tailgate in their vehicles. The presence of tables with food, drinks, and decor

In [16]:
vid_set = videos[-8]
print(vid_set.keys())
print(vid_set["vid_path"])
for key in vid_set.keys():
    if("vT" not in key):
        print(key, vid_set[key])

dict_keys(['vid_path', 'vT0', 'vT1', 'vT2', 'T0_description', 'T1_description', 'T2_description', 'T0_timestep', 'T1_timestep', 'T2_timestep', 'T0_qa', 'T1_qa', 'T2_qa'])
/home/ubuntu/temp/10k_vid_dataset/mp4s/video_1492.mp4
vid_path /home/ubuntu/temp/10k_vid_dataset/mp4s/video_1492.mp4
T0_description {'Q': 'What occurs between 0 and 10 seconds?', 'A': 'The video begins with a close-up of a red and white shield logo, likely for a sports team or event, featuring the name "Scotiabank" prominently. The scene then transitions to a wide shot of a soccer stadium where a group of players in different colored jerseys, including red, blue, and yellow, are lined up across the field, walking in formation. This appears to be a pre-match ceremony, possibly the national anthems or team introductions. In the background, spectators are seated in the stands, and some are wearing masks. A flag with a star and a red and white design is visible in the foreground, which seems to be related to Puerto Rico, 

In [17]:
#save to huggingface dictionary
from datasets import Dataset
root_path = "/home/ubuntu/temp/10k_vid_dataset/"
#recreate data list without video frames
data_list = []
for vid in videos:
    vid_dict = {}
    for key in vid.keys():
        if("vT" not in key):
            vid_dict[key] = vid[key]
    data_list.append(vid_dict)
#save to huggingface dataset
dataset = Dataset.from_list(data_list)
dataset.save_to_disk(os.path.join(root_path, "finevideo_dataset"))

Saving the dataset (0/1 shards):   0%|          | 0/1452 [00:00<?, ? examples/s]

In [18]:
test_load = Dataset.load_from_disk(os.path.join(root_path, "finevideo_dataset"))
print(test_load[0].keys())

dict_keys(['vid_path', 'T0_description', 'T1_description', 'T2_description', 'T0_timestep', 'T1_timestep', 'T2_timestep', 'T0_qa', 'T1_qa', 'T2_qa'])


In [19]:
len(test_load)

1452